<a href="https://colab.research.google.com/github/cowsilver57/sessac_test/blob/main/Chapter6_%EA%B8%B0%EC%B6%9C3%ED%9A%8C_2%EC%9C%A0%ED%98%95(18%EC%9D%BC%EC%B0%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#github와 colab 연동
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

fatal: destination path 'yemoonsaBigdata' already exists and is not an empty directory.


다음은 Travel Insurance 데이터 세트이다.

1-1. 주어진 훈련 데이터 세트를 이용하여 고객별 여행보험 가입 여부 예측 모형을 만들고,

1-2.가장 높은 Accuracy 값을 가지는 최종 모델을 도출하시오.

1-3.해당 모델을 활용하여 보험 가입 여부 예측 값을 계산하고 결괏값은 csv 파일로 제출하시오.

In [ ]:
import pandas as pd


X_test = pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/304_x_test.csv')
X_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/304_x_train.csv')
y_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/304_y_train.csv')

## 데이터 확인하기

In [ ]:
X_test.head()

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,1569,27,Government Sector,Yes,500000,5,0,No,No
1,1344,25,Private Sector/Self Employed,Yes,1700000,3,0,Yes,No
2,1429,32,Government Sector,Yes,650000,3,0,No,No
3,896,33,Government Sector,Yes,600000,4,0,No,No
4,101,33,Private Sector/Self Employed,Yes,1500000,3,1,Yes,Yes


In [ ]:
X_train.head()

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,1704,26,Private Sector/Self Employed,Yes,1400000,3,1,No,Yes
1,491,28,Private Sector/Self Employed,Yes,1100000,4,1,No,No
2,414,33,Private Sector/Self Employed,Yes,1400000,4,0,No,Yes
3,120,28,Private Sector/Self Employed,Yes,800000,3,1,No,No
4,1268,33,Government Sector,Yes,1000000,5,0,No,Yes


In [ ]:
y_train.head()

,ID,TravelInsurance
0,1704,0
1,491,1
2,414,1
3,120,0
4,1268,1


## 데이터 정보 확인하기

In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   497 non-null    int64 
 1   Age                  497 non-null    int64 
 2   Employment Type      497 non-null    object
 3   GraduateOrNot        497 non-null    object
 4   AnnualIncome         497 non-null    int64 
 5   FamilyMembers        497 non-null    int64 
 6   ChronicDiseases      497 non-null    int64 
 7   FrequentFlyer        497 non-null    object
 8   EverTravelledAbroad  497 non-null    object
dtypes: int64(5), object(4)
memory usage: 35.1+ KB


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
dtypes: int64(5), object(4)
memory usage: 104.9+ KB


In [ ]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   ID               1490 non-null   int64
 1   TravelInsurance  1490 non-null   int64
dtypes: int64(2)
memory usage: 23.4 KB


## 결측치 확인하기

In [ ]:
#결측치 확인하기
X_test.isnull().sum()

ID                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [ ]:
X_train.isnull().sum()

ID                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [ ]:
y_train.isnull().sum()

ID                 0
TravelInsurance    0
dtype: int64

## 변수 분류하기

In [ ]:
COL_DEL = ['ID']
COL_NUM = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
COL_CAT = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']
COL_Y = ['TravelInsurance']

## 데이터 분할하기

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train[COL_NUM+COL_CAT],
                                            y_train[COL_Y].values.ravel(),
                                            test_size=0.3,
                                            stratify=y_train[COL_Y].values.ravel())
print(X_tr.head())

      Age  AnnualIncome  FamilyMembers  ChronicDiseases  Employment Type  \
1276   28       1150000              7                0                1   
531    30        650000              3                1                1   
1247   28       1450000              5                0                1   
1448   31       1250000              3                1                1   
1347   34        700000              3                0                1   

      GraduateOrNot  FrequentFlyer  EverTravelledAbroad  
1276              1              0                    0  
531               0              0                    0  
1247              1              1                    1  
1448              1              0                    0  
1347              1              1                    0  


In [ ]:
print(y_tr)

[0 0 1 ... 1 0 0]


## 데이터 전처리

In [ ]:
#수치형 데이터 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])

X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [ ]:
print(X_tr[COL_NUM].head())

           Age  AnnualIncome  FamilyMembers  ChronicDiseases
1276 -0.594941      0.608352       1.432206        -0.647339
531   0.089586     -0.731410      -1.094290         1.544786
1247 -0.594941      1.412210       0.168958        -0.647339
1448  0.431849      0.876305      -1.094290         1.544786
1347  1.458640     -0.597434      -1.094290        -0.647339


In [ ]:
from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train[COL_CAT], X_test[COL_CAT]])

for col in COL_CAT:
  le = LabelEncoder()
  le.fit(X[col])

  X_tr[col] = le.transform(X_tr[col])
  X_val[col] = le.transform(X_val[col])
  X_test[col] = le.transform(X_test[col])

  print(col)
  print(le.classes_)
  print('\n')

Employment Type
[0 1]


GraduateOrNot
[0 1]


FrequentFlyer
[0 1]


EverTravelledAbroad
[0 1]




In [ ]:
print(X_tr[COL_CAT].head())

      Employment Type  GraduateOrNot  FrequentFlyer  EverTravelledAbroad
1276                1              1              0                    0
531                 1              0              0                    0
1247                1              1              1                    1
1448                1              1              0                    0
1347                1              1              1                    0


## 모델 학습시키기

In [ ]:
#모델 학습시키기
#1. 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(random_state=123)
modelRF.fit(X_tr, y_tr)

RandomForestClassifier(random_state=123)

In [ ]:
#모델 학습시키기
#2. XGBoost
from xgboost import XGBClassifier
modelXGB = XGBClassifier(random_state=123)
modelXGB.fit(X_tr, y_tr)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=123, ...)

## 모델 예측값 생성 및 평가 지표 구하기

In [ ]:
#모델 예측값 생성
y_val_predRF = modelRF.predict_proba(X_val)[:,1]
y_val_predXGB = modelXGB.predict_proba(X_val)[:,1]

In [ ]:
#평가 지표 구하기
from sklearn.metrics import roc_auc_score
scoreRF = roc_auc_score(y_val, y_val_predRF)
scoreXGB = roc_auc_score(y_val, y_val_predXGB)

print("RandomFores:", scoreRF)
print("XGBoost:", scoreXGB)
#XGBoost로 모델 결정!

RandomFores: 0.8011587611002815
XGBoost: 0.8030864197530865


## 학습, 검증 성능 확인 함수

In [ ]:
#학습, 검증 성능 확인 함수
def get_scores(model, C_tr, X_val, y_tr, y_val):
    y_tr_pred = model.predict_proba(X_tr)[:,1]
    y_val_pred = model.predict_proba(X_val)[:,1]
    tr_score = roc_auc_score(y_tr, y_tr_pred)
    val_score = roc_auc_score(y_val, y_val_pred)
    return f'train: {round(tr_score,4)}, valid:{round(val_score,4)}'

print('Random Forest:', get_scores(modelRF, X_tr, X_val, y_tr, y_val))
print('XGBoost:', get_scores(modelXGB, X_tr, X_val, y_tr, y_val))

Random Forest: train: 0.9909, valid:0.8012
XGBoost: train: 0.9794, valid:0.8031


In [ ]:
#성능 개선을 위한 XGBoost의 파라미터 조정
modelXGB2 = XGBClassifier(n_estimators=50, max_depth=3, min_child_weight=1, random_state=123)
modelXGB2.fit(X_tr, y_tr)
modelXGB3 = XGBClassifier(n_estimators=50, max_depth=5, min_child_weight=1, random_state=123)
modelXGB3.fit(X_tr, y_tr)
modelXGB4 = XGBClassifier(n_estimators=50, max_depth=3, min_child_weight=2, random_state=123)
modelXGB4.fit(X_tr, y_tr)
modelXGB5 = XGBClassifier(n_estimators=50, max_depth=5, min_child_weight=2, random_state=123)
modelXGB5.fit(X_tr, y_tr)

print('XGBoost2:', get_scores(modelXGB2, X_tr, X_val, y_tr, y_val))
print('XGBoost3:', get_scores(modelXGB3, X_tr, X_val, y_tr, y_val))
print('XGBoost4:', get_scores(modelXGB4, X_tr, X_val, y_tr, y_val))
print('XGBoost5:', get_scores(modelXGB5, X_tr, X_val, y_tr, y_val))

#valid는 XGBoost5가 0.839로 가장 높음. -> 성능이 가장 좋음.

XGBoost2: train: 0.8982, valid:0.8199
XGBoost3: train: 0.9452, valid:0.8153
XGBoost4: train: 0.8795, valid:0.8193
XGBoost5: train: 0.9271, valid:0.8204


In [ ]:
pred = modelXGB2.predict_proba(X_test[COL_NUM + COL_CAT])[:, 1]
result = pd.DataFrame({'ID':X_test.ID, 'pred':pred})
print(result)

       ID      pred
0    1569  0.026379
1    1344  0.020906
2    1429  0.026379
3     896  0.026379
4     101  0.272680
..    ...       ...
492  1376  0.322362
493    87  0.026379
494   287  0.026379
495   337  0.009435
496    92  0.026379

[497 rows x 2 columns]


In [ ]:
result.to_csv('00300000.csv',index=False)